In [4]:
import pickle
import pandas as pd
import matplotlib
import os
import re

import scipy

import collections
import datetime
import time

import geopandas as gpd

import numpy as np
 
from difflib import get_close_matches

from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from sklearn import linear_model
import statsmodels.api as sm

from linearmodels import PanelOLS, FamaMacBeth
from scipy import stats

import itertools


## Grab Data

In [8]:
os.getcwd()

'/Users/brianreed/Documents/supplyChain/extremes/extremesAnalysisCode'

In [87]:
# goodsData = pd.read_csv("../goodsData_suppliers.csv").drop(columns = {'Unnamed: 0'})
goodsData = pd.read_csv("../goodsData_igData.csv").drop(columns = {'Unnamed: 0'})
# goodsData = pd.read_csv("../utilitiesData_igData.csv").drop(columns = {'Unnamed: 0'})

# goodsData = pd.read_csv("../goodsData.csv").drop(columns = {'Unnamed: 0'})
goodsData = goodsData[~goodsData.revenueChange.isna() & 
                     ~goodsData.costChange.isna()]

# goodsData['precip5Days_annualquant_1x5Yrs'] = 1*(goodsData['precip5Days_annualquant_1x5Yrs'] > 0)

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# goodsData

Let's grab a list with all the relevant variable names. Just keep the core names and we'll grab the lags as we go through it.

Look at the full combinations of variables and whatnot, if we'd like.

In [50]:
'''
print(goodsData.columns[0:50])

cutoffVars  = ['0.95','1xQtr','1xYr','1x5Qtrs','1x5Yrs'] # ,'1x10Qtrs','1x10Yrs'
weatherVars = ['precip_','temp_','precip5Days_','temp5Days_']
statVars    = ['annualquant_','zipquant_','zipQuarterquant_']

outcomeVars = ['lnCost','lnRev','lnCostNormd','lnRevNormd']
# ['incomeChange', 'revenueChange', 'costChange'],'lnCost','lnInc','lnRev']
# ['lnCost','lnRev','lnCostNormd','lnRevNormd']


for weatherVar in weatherVars:
    for statVar in statVars:
        for cutoffVar in cutoffVars:
            indVar = weatherVar + statVar + cutoffVar
            
            for outcomeVar in outcomeVars:
                print(indVar,"~", outcomeVar)'''

'\nprint(goodsData.columns[0:50])\n\ncutoffVars  = [\'0.95\',\'1xQtr\',\'1xYr\',\'1x5Qtrs\',\'1x5Yrs\'] # ,\'1x10Qtrs\',\'1x10Yrs\'\nweatherVars = [\'precip_\',\'temp_\',\'precip5Days_\',\'temp5Days_\']\nstatVars    = [\'annualquant_\',\'zipquant_\',\'zipQuarterquant_\']\n\noutcomeVars = [\'lnCost\',\'lnRev\',\'lnCostNormd\',\'lnRevNormd\']\n# [\'incomeChange\', \'revenueChange\', \'costChange\'],\'lnCost\',\'lnInc\',\'lnRev\']\n# [\'lnCost\',\'lnRev\',\'lnCostNormd\',\'lnRevNormd\']\n\n\nfor weatherVar in weatherVars:\n    for statVar in statVars:\n        for cutoffVar in cutoffVars:\n            indVar = weatherVar + statVar + cutoffVar\n            \n            for outcomeVar in outcomeVars:\n                print(indVar,"~", outcomeVar)'

Focus on the 5 day variables now (we'll later need to edit the loops above to account for this as well, but we can do this for now.) Make them 0/1.

# Direct Effects
Look at the effects on the suppliers when they're affected directly.

The below gives us the full, clustered standard errors.

In [92]:
cutoffVarsYr = ['0.95']  #, ] # ,'1xQtr''1x5Qtrs',
weatherVars  = ['precip_', 'temp_'] #, 'temp5Days_', 'precip5Days_'] # , 'precip_']#, , ] #[,]
statVarsYr   = ['zipquant_', 'ffquant_', 'zipQuarterquant_', 'indQuarterquant_'] #  , , ]  #,'zipQuarterquant_']
outcomeVars  = ['lnRevNormd', 'lnCostNormd', 'lnRev', 'lnCost', 'revenueChange', 'costChange']

goodsData = goodsData[~goodsData.lnRev.isna() & ~goodsData.lnCost.isna()] # & ~goodsData.lnCostNormd.isna()]


start = time.time()

results = pd.DataFrame()

i = 0
for outcomeVar in outcomeVars:
    for weatherVar in weatherVars:
        for statVar in statVarsYr:                     
            for cutoffVar in cutoffVarsYr:
                i = i + 1
                indVar = weatherVar + statVar + cutoffVar
                
                
                print(outcomeVar, "~", indVar)


                # find: concurrent ; or lagged supplier data
                X = goodsData.loc[:,((goodsData.columns.str.contains(indVar)) | 
                                                (goodsData.columns.str.contains('indQtr_')) |
                                                (goodsData.columns.str.contains('gvkey_'))  | 
                                                # (goodsData.columns.str.contains('firmQtr_')) |
                                                (goodsData.columns.str.contains('ageQtr_')) |
                                                (goodsData.columns.str.contains('sizeQtr_')) |
                                                (goodsData.columns.str.contains('profitQtr_')))]
                
                
                firms = goodsData['gvkey']


                y = goodsData[outcomeVar]
                
                
                model = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)
                pvals = model.pvalues[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]
                coeff =  model.params[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]
                
                # results.loc[i,'ind'] = ind


                results.loc[i,'outcomeVar'] = outcomeVar
                results.loc[i,'weatherVar'] = weatherVar
                results.loc[i,'statVar']    = statVar
                results.loc[i,'cutoffVar']  = cutoffVar
                # results.loc[i,'firmVar']    = firmVar


                results.loc[i,'lag0']       = coeff[0]
                results.loc[i,'lag1']       = coeff[1]
                results.loc[i,'lag2']       = coeff[2]
                results.loc[i,'lag3']       = coeff[3]
                results.loc[i,'lag4']       = coeff[4]

                results.loc[i,'pval0']      = pvals[0]
                results.loc[i,'pval1']      = pvals[1]
                results.loc[i,'pval2']      = pvals[2]
                results.loc[i,'pval3']      = pvals[3]
                results.loc[i,'pval4']      = pvals[4]

                
                results.to_csv("../../data/utilitiesResults_rightInds.csv")
                
                print( time.time() - start)

lnRevNormd ~ precip_zipquant_0.95
1678.5230588912964
lnRevNormd ~ precip_ffquant_0.95
3392.479749917984
lnRevNormd ~ precip_zipQuarterquant_0.95
6141.696490764618
lnRevNormd ~ precip_indQuarterquant_0.95
7792.087285757065
lnRevNormd ~ temp_zipquant_0.95
9536.280804872513
lnRevNormd ~ temp_ffquant_0.95
11397.445341825485
lnRevNormd ~ temp_zipQuarterquant_0.95
13390.505018949509
lnRevNormd ~ temp_indQuarterquant_0.95
15205.601019859314
lnCostNormd ~ precip_zipquant_0.95
17386.848616838455
lnCostNormd ~ precip_ffquant_0.95
19263.92868590355
lnCostNormd ~ precip_zipQuarterquant_0.95
21002.800498008728
lnCostNormd ~ precip_indQuarterquant_0.95
23055.123690843582
lnCostNormd ~ temp_zipquant_0.95
24807.84149980545
lnCostNormd ~ temp_ffquant_0.95
26550.97590303421
lnCostNormd ~ temp_zipQuarterquant_0.95
28377.4769449234
lnCostNormd ~ temp_indQuarterquant_0.95
30363.80248785019
lnRev ~ precip_zipquant_0.95
32149.090194940567
lnRev ~ precip_ffquant_0.95
34005.434716939926
lnRev ~ precip_zipQuart

KeyboardInterrupt: 

In [54]:
results.to_csv("../../data/utilitiesResults_rightInds.csv")

## Industry-Specific
Go through every famafrench industry and run the regressions above.

In [12]:
cutoffVarsYr = ['0.95']  #, ] # ,'1xQtr''1x5Qtrs',
weatherVars  = ['precip_', 'temp_'] #, 'temp5Days_', 'precip5Days_'] # , 'precip_']#, , ] #[,]
statVarsYr   = ['zipquant_', 'ffquant_', 'zipQuarterquant_', 'indQuarterquant_'] #  , , ]  #,'zipQuarterquant_']
outcomeVars  = ['lnRevNormd', 'lnCostNormd', 'lnRev', 'lnCost', 'revenueChange', 'costChange']

industries = range(1,44)

start = time.time()

results = pd.DataFrame()

i = 0
for ind in industries:
    print('##########################################################')
    print(ind)
    filename = '../../data/companyData/igData_ind' + str(ind) + '.csv'           
    goodsData = pd.read_csv(filename).drop(columns = {'Unnamed: 0'})

    
    for outcomeVar in outcomeVars:
        for weatherVar in weatherVars:
            for statVar in statVarsYr:                     
                for cutoffVar in cutoffVarsYr:
                    
                    i = i + 1
                    
                    
                    
                    goodsData = goodsData[~goodsData.revenueChange.isna() & 
                                         ~goodsData.costChange.isna() & 
                                         ~goodsData.lnRev.isna() & 
                                         ~goodsData.lnCost.isna()]
                    
                    
                    indVar = weatherVar + statVar + cutoffVar


                    print(outcomeVar, "~", indVar)


                    # find: concurrent ; or lagged supplier data
                    X = goodsData.loc[:,((goodsData.columns.str.contains(indVar)) | 
                                                    (goodsData.columns.str.contains('indQtr_')) |
                                                    (goodsData.columns.str.contains('gvkey_'))  | 
                                                    (goodsData.columns.str.contains('ageQtr_')) |
                                                    (goodsData.columns.str.contains('sizeQtr_')) |
                                                    (goodsData.columns.str.contains('profitQtr_')))]


                    firms = goodsData['gvkey']


                    y = goodsData[outcomeVar]


                    model = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)
                    pvals = model.pvalues[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]
                    coeff =  model.params[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]

                    # results.loc[i,'ind'] = ind

                    results.loc[i,'industry'] = ind

                    results.loc[i,'outcomeVar'] = outcomeVar
                    results.loc[i,'weatherVar'] = weatherVar
                    results.loc[i,'statVar']    = statVar
                    results.loc[i,'cutoffVar']  = cutoffVar
                    # results.loc[i,'firmVar']    = firmVar


                    results.loc[i,'lag0']       = coeff[0]
                    results.loc[i,'lag1']       = coeff[1]
                    results.loc[i,'lag2']       = coeff[2]
                    results.loc[i,'lag3']       = coeff[3]
                    results.loc[i,'lag4']       = coeff[4]

                    results.loc[i,'pval0']      = pvals[0]
                    results.loc[i,'pval1']      = pvals[1]
                    results.loc[i,'pval2']      = pvals[2]
                    results.loc[i,'pval3']      = pvals[3]
                    results.loc[i,'pval4']      = pvals[4]


                    results.to_csv("../../results_byInds.csv")

                    print( time.time() - start)

##########################################################
1
lnRevNormd ~ precip_zipquant_0.95
0.3025510311126709
lnRevNormd ~ precip_ffquant_0.95
0.7997560501098633
lnRevNormd ~ precip_zipQuarterquant_0.95
1.0998473167419434
lnRevNormd ~ precip_indQuarterquant_0.95
1.383890151977539
lnRevNormd ~ temp_zipquant_0.95
1.6102750301361084
lnRevNormd ~ temp_ffquant_0.95
2.0266880989074707
lnRevNormd ~ temp_zipQuarterquant_0.95
2.2949953079223633
lnRevNormd ~ temp_indQuarterquant_0.95
2.5350961685180664
lnCostNormd ~ precip_zipquant_0.95
2.8127002716064453
lnCostNormd ~ precip_ffquant_0.95
3.1637701988220215
lnCostNormd ~ precip_zipQuarterquant_0.95
3.5269880294799805
lnCostNormd ~ precip_indQuarterquant_0.95
3.834968090057373
lnCostNormd ~ temp_zipquant_0.95
4.131942272186279
lnCostNormd ~ temp_ffquant_0.95
4.675650119781494
lnCostNormd ~ temp_zipQuarterquant_0.95
4.989803075790405
lnCostNormd ~ temp_indQuarterquant_0.95
5.614145040512085
lnRev ~ precip_zipquant_0.95
5.990991115570068
lnRev 

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


lnRevNormd ~ precip_zipquant_0.95
17.66443634033203
lnRevNormd ~ precip_ffquant_0.95
19.169827222824097
lnRevNormd ~ precip_zipQuarterquant_0.95
20.68500828742981
lnRevNormd ~ precip_indQuarterquant_0.95
21.938390016555786
lnRevNormd ~ temp_zipquant_0.95
23.43198013305664
lnRevNormd ~ temp_ffquant_0.95
25.11342215538025
lnRevNormd ~ temp_zipQuarterquant_0.95
26.356121063232422
lnRevNormd ~ temp_indQuarterquant_0.95
27.739347219467163
lnCostNormd ~ precip_zipquant_0.95
29.14668321609497
lnCostNormd ~ precip_ffquant_0.95
30.479042053222656
lnCostNormd ~ precip_zipQuarterquant_0.95
31.72460412979126
lnCostNormd ~ precip_indQuarterquant_0.95
33.05599117279053
lnCostNormd ~ temp_zipquant_0.95
34.46054220199585
lnCostNormd ~ temp_ffquant_0.95
35.95199418067932
lnCostNormd ~ temp_zipQuarterquant_0.95
37.2239453792572
lnCostNormd ~ temp_indQuarterquant_0.95
38.4719512462616
lnRev ~ precip_zipquant_0.95
39.8163321018219
lnRev ~ precip_ffquant_0.95
41.232704162597656
lnRev ~ precip_zipQuarterqua

151.19423413276672
lnRevNormd ~ temp_zipQuarterquant_0.95
151.22602796554565
lnRevNormd ~ temp_indQuarterquant_0.95
151.31181621551514
lnCostNormd ~ precip_zipquant_0.95
151.39567613601685
lnCostNormd ~ precip_ffquant_0.95
151.42706727981567
lnCostNormd ~ precip_zipQuarterquant_0.95
151.4682171344757
lnCostNormd ~ precip_indQuarterquant_0.95
151.50130820274353
lnCostNormd ~ temp_zipquant_0.95
151.5324001312256
lnCostNormd ~ temp_ffquant_0.95
151.56680130958557
lnCostNormd ~ temp_zipQuarterquant_0.95
151.59654712677002
lnCostNormd ~ temp_indQuarterquant_0.95
151.62548398971558
lnRev ~ precip_zipquant_0.95
151.69387531280518
lnRev ~ precip_ffquant_0.95
151.72398900985718
lnRev ~ precip_zipQuarterquant_0.95
151.75547909736633
lnRev ~ precip_indQuarterquant_0.95
151.79284501075745
lnRev ~ temp_zipquant_0.95
151.82540726661682
lnRev ~ temp_ffquant_0.95
151.85733222961426
lnRev ~ temp_zipQuarterquant_0.95
151.8889582157135
lnRev ~ temp_indQuarterquant_0.95
151.9383201599121
lnCost ~ precip_z

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

lnRevNormd ~ precip_zipquant_0.95
2194.2858102321625
lnRevNormd ~ precip_ffquant_0.95
2196.4031360149384
lnRevNormd ~ precip_zipQuarterquant_0.95
2198.3404412269592
lnRevNormd ~ precip_indQuarterquant_0.95
2200.475779056549
lnRevNormd ~ temp_zipquant_0.95
2202.478760242462
lnRevNormd ~ temp_ffquant_0.95
2204.5217871665955
lnRevNormd ~ temp_zipQuarterquant_0.95
2206.489814043045
lnRevNormd ~ temp_indQuarterquant_0.95
2208.541850090027
lnCostNormd ~ precip_zipquant_0.95
2210.5746371746063
lnCostNormd ~ precip_ffquant_0.95
2212.7245922088623
lnCostNormd ~ precip_zipQuarterquant_0.95
2214.7043051719666
lnCostNormd ~ precip_indQuarterquant_0.95
2216.6849641799927
lnCostNormd ~ temp_zipquant_0.95
2218.740087032318
lnCostNormd ~ temp_ffquant_0.95
2220.8818180561066
lnCostNormd ~ temp_zipQuarterquant_0.95
2222.881567955017
lnCostNormd ~ temp_indQuarterquant_0.95
2224.893409013748
lnRev ~ precip_zipquant_0.95
2226.981845140457
lnRev ~ precip_ffquant_0.95
2229.082554101944
lnRev ~ precip_zipQuar

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2303.5703060626984
lnRevNormd ~ precip_ffquant_0.95
2305.5923392772675
lnRevNormd ~ precip_zipQuarterquant_0.95
2307.1460993289948
lnRevNormd ~ precip_indQuarterquant_0.95
2308.884642124176
lnRevNormd ~ temp_zipquant_0.95
2310.704126358032
lnRevNormd ~ temp_ffquant_0.95
2312.322122335434
lnRevNormd ~ temp_zipQuarterquant_0.95
2313.849548101425
lnRevNormd ~ temp_indQuarterquant_0.95
2315.518447160721
lnCostNormd ~ precip_zipquant_0.95
2317.0042123794556
lnCostNormd ~ precip_ffquant_0.95
2318.4067792892456
lnCostNormd ~ precip_zipQuarterquant_0.95
2319.9825382232666
lnCostNormd ~ precip_indQuarterquant_0.95
2321.3990530967712
lnCostNormd ~ temp_zipquant_0.95
2323.9209752082825
lnCostNormd ~ temp_ffquant_0.95
2327.0702431201935
lnCostNormd ~ temp_zipQuarterquant_0.95
2329.9983212947845
lnCostNormd ~ temp_indQuarterquant_0.95
2331.7974030971527
lnRev ~ precip_zipquant_0.95
2333.4375143051147
lnRev ~ precip_ffquant_0.95
2334.9315021038055
lnRev ~ precip_zipQuarterquant_0.95
2336.85118007659

2563.1340873241425
lnRevNormd ~ temp_zipQuarterquant_0.95
2565.4282932281494
lnRevNormd ~ temp_indQuarterquant_0.95
2567.164327144623
lnCostNormd ~ precip_zipquant_0.95
2568.80464720726
lnCostNormd ~ precip_ffquant_0.95
2570.753844976425
lnCostNormd ~ precip_zipQuarterquant_0.95
2572.4220242500305
lnCostNormd ~ precip_indQuarterquant_0.95
2574.0690972805023
lnCostNormd ~ temp_zipquant_0.95
2575.5401163101196
lnCostNormd ~ temp_ffquant_0.95
2577.4163041114807
lnCostNormd ~ temp_zipQuarterquant_0.95
2579.5036101341248
lnCostNormd ~ temp_indQuarterquant_0.95
2580.6666481494904
lnRev ~ precip_zipquant_0.95
2582.288702249527
lnRev ~ precip_ffquant_0.95
2583.7519233226776
lnRev ~ precip_zipQuarterquant_0.95
2585.307689189911
lnRev ~ precip_indQuarterquant_0.95
2586.654230117798
lnRev ~ temp_zipquant_0.95
2587.524547100067
lnRev ~ temp_ffquant_0.95
2589.089822292328
lnRev ~ temp_zipQuarterquant_0.95
2590.327291250229
lnRev ~ temp_indQuarterquant_0.95
2591.550584077835
lnCost ~ precip_zipquant

LinAlgError: SVD did not converge

# Faster and More Heuristic

The below gives us unclustered standard errors, output to a csv file.

In [ ]:
def findSE(X,reg,y):
    N = len(X)
    p = len(X.columns) + 1  # plus one because LinearRegression adds an intercept term

    X_with_intercept = np.empty(shape=(N, p), dtype=np.float)
    X_with_intercept[:, 0] = 1
    X_with_intercept[:, 1:p] = X.values

    y_hat = reg.predict(X)
    residuals = y.values - y_hat
    residual_sum_of_squares = residuals.T @ residuals
    sigma_squared_hat = residual_sum_of_squares / (N - p)
    var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat

    se0 = var_beta_hat[1, 1] ** 0.5
    se1 = var_beta_hat[2, 2] ** 0.5
    se2 = var_beta_hat[3, 3] ** 0.5
    se3 = var_beta_hat[4, 4] ** 0.5
    se4 = var_beta_hat[5, 5] ** 0.5
    se5 = var_beta_hat[6, 6] ** 0.5
    '''se6 = var_beta_hat[7, 7] ** 0.5
    se7 = var_beta_hat[8, 8] ** 0.5
    se8 = var_beta_hat[9, 9] ** 0.5'''
    return([abs(reg.coef_[0]/se0),abs(reg.coef_[1]/se1),abs(reg.coef_[2]/se2),
            abs(reg.coef_[3]/se3),abs(reg.coef_[4]/se4),abs(reg.coef_[5]/se5)]
          )

'''        
abs(reg.coef_[0]/se0),
          abs(reg.coef_[1]/se1),
          abs(reg.coef_[2]/se2),
          abs(reg.coef_[3]/se3),
          abs(reg.coef_[4]/se4),
          abs(reg.coef_[5]/se5),
          "SE0: ", se0,
          "SE1: ", se1,
          "SE2: ", se2,
          "SE3: ", se3,
          "SE4: ", se4,
          "SE5: ", se5,

'''


In [ ]:
goodsData.columns[0:75]

In [ ]:
goodsData.columns[0:50]

In [ ]:
inds = [1, 2, 6, 7, 18, 31, 41, 42]

for ind in inds:

    tempData = goodsData[goodsData.famafrench == ind]
    print(tempData.shape)


In [ ]:
'''cutoffVarsYr = ['0.95'] # ,'1xYr']                                    #,'1x5Yrs'] #, ] # ,'1xQtr', '1x5Qtrs'
weatherVars  = ['precip_', 'temp_', 'precip5Days_', 'temp5Days_'] #[,]
statVarsYr   = ['zipquant_','zipQuarterquant_']
outcomeVars  = ['lnRev', 'revenueChange'] # ,'lnCost',  'costChange'] # [,'lnRevNormd','lnCostNormd'] # 'revenueChange' 'costChange',
firmVars     = ['firmQtr_'] # 'gvkey'
'''

# try this by industry
cutoffVarsYr = ['0.95'] # ,'1xYr']                                    #,'1x5Yrs'] #, ] # ,'1xQtr', '1x5Qtrs'
weatherVars  = ['precip_', 'temp_', 'precip5Days_', 'temp5Days_'] #[,]
statVarsYr   = ['ffquant_','indQuarterquant_']
outcomeVars  = ['lnRev', 'revenueChange',  'lnCost',  'costChange'] # [,'lnRevNormd','lnCostNormd'] # 'revenueChange' 'costChange',
firmVars     = ['firmQtr_']


inds = [1, 2, 6, 7, 18, 31, 41, 42]

goodsData = goodsData[~goodsData.lnRev.isna() & ~goodsData.lnCost.isna() &
                      ~goodsData.lnCostNormd.isna() & ~goodsData.lnRevNormd.isna()]

start = time.time()

results = pd.DataFrame()
i = 0
for ind in inds:
    print('#######################################################################################',ind)
    for outcomeVar in outcomeVars:
        for weatherVar in weatherVars:
            for statVar in statVarsYr:                     
                for cutoffVar in cutoffVarsYr:
                    for firmVar in firmVars:
                        tempData = goodsData[goodsData.famafrench == ind]
                        
                        i = i + 1
                        indVar = weatherVar + statVar + cutoffVar


                        print(outcomeVar, "~", indVar, "|", firmVar)


                        # find: concurrent ; or lagged supplier data
                        X = tempData.loc[:,((tempData.columns.str.contains(indVar)) |
                                          (tempData.columns.str.contains('indQtr_')) |
                                          # (goodsData.columns.str.contains('gvkey_'))) |   # &   
                                          # (goodsData.columns.str.contains('firmQtr_'))) |
                                          (tempData.columns.str.contains(firmVar)))] # |
                        '''(tempData.columns.str.contains('ageQtr_')) |
                          (tempData.columns.str.contains('sizeQtr_')) |
                          (tempData.columns.str.contains('profitQtr_'))]   #  & '''

                                          # (goodsData.columns.str.contains('firmQtr_')))       & 
                                        # ~(goodsData.columns.str.contains('lag4')) &
                                                                        # ~(goodsData.columns.str.contains('lag2')) & 


                        X = X[X.columns[(X.sum(axis = 0) >= 4)]]
                        # print(X.columns)
                        firms = tempData['gvkey']


                        y = tempData[outcomeVar]


                        ######################################
                        # fit the model on this subset
                        reg = linear_model.LinearRegression()
                        reg.fit(X,y)


                        # print('Coeff: ' , reg.coef_[0:5], 'SE type (looking >2): ', findSE(X,reg,y))
                        results.loc[i,'ind'] = ind


                        results.loc[i,'outcomeVar'] = outcomeVar
                        results.loc[i,'weatherVar'] = weatherVar
                        results.loc[i,'statVar']    = statVar
                        results.loc[i,'cutoffVar']  = cutoffVar
                        results.loc[i,'firmVar']    = firmVar


                        results.loc[i,'lag0']       = reg.coef_[0]
                        results.loc[i,'lag1']       = reg.coef_[1]
                        results.loc[i,'lag2']       = reg.coef_[2]
                        results.loc[i,'lag3']       = reg.coef_[3]
                        results.loc[i,'lag4']       = reg.coef_[4]



                        seratios = findSE(X,reg,y)

                        results.loc[i,'ratio0']       = seratios[0]
                        results.loc[i,'ratio1']       = seratios[1]
                        results.loc[i,'ratio2']       = seratios[2]
                        results.loc[i,'ratio3']       = seratios[3]
                        results.loc[i,'ratio4']       = seratios[4]

                        # print(results)

                        print(time.time() - start)

                        print('*******************************************************************')
                    
results.to_csv("../../data/results_notNormd.csv")

In [ ]:
i = 0

results = pd.DataFrame()
results.loc[i,'outcome'] = 0
results

# Indirect Effects
This is almost exactly the same but with supplier information in place of the direct company information.

In [ ]:
cutoffVarsYr = ['1xYr'] #, ] # ,'1xQtr''1x5Qtrs',
weatherVars  = ['precip_', 'temp_', 'precip5Days_', 'temp5Days_'] #[,]
statVarsYr   = ['annualquant_']  #,'zipQuarterquant_']
outcomeVars  = ['costChange', 'revenueChange', 'lnRevNormd', 'lnCostNormd'] # ['revenueChange'] #[, 'costChange']#,'lnCost','lnInc','lnRev']

goodsData = goodsData[~goodsData.lnRev.isna() & ~goodsData.lnCost.isna() & ~goodsData.lnCostNormd.isna()]
goodsData['scTercile']  = pd.qcut(goodsData['suppliers'], 3, labels=False, duplicates = 'drop')


start = time.time()

for outcomeVar in outcomeVars:
    for weatherVar in weatherVars:
        for statVar in statVarsYr:                     
            for cutoffVar in cutoffVarsYr:
                indVar = weatherVar + statVar + cutoffVar
                
                
                print(outcomeVar, "~", indVar)


                # find: concurrent ; or lagged supplier data
                X = goodsData.loc[:,(((goodsData.columns.str.contains(indVar)) & goodsData.columns.str.contains('supplier')) | 
                                                (goodsData.columns.str.contains('indQtr_')) |
                                                (goodsData.columns.str.contains('gvkey_'))  | 
                                                (goodsData.columns.str.contains('ageQtr_')) |
                                                (goodsData.columns.str.contains('sizeQtr_')) |
                                                (goodsData.columns.str.contains('profitQtr_')) | 
                                                (goodsData.columns == 'suppliers')) &   
                                                # (goodsData.columns.str.contains('firmQtr_')))       & 
                                                ~(goodsData.columns.str.contains('gvkey_1013')) & # do 1 minus the FEs
                                                ~(goodsData.columns.str.contains('indQtr_12010_1'))]# & 
                                                # ~(goodsData.columns.str.contains('lag4'))]     
                
                
                firms = goodsData['gvkey']


                y = goodsData[outcomeVar]
                
                
                results = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)
                pvals = results.pvalues[0:len(goodsData.columns[goodsData.columns.str.contains(indVar) & goodsData.columns.str.contains('supplier_')])]
                coeff = results.params[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)  & goodsData.columns.str.contains('supplier_')])]
                print(coeff)
                print(pvals)
                
                
                print( time.time() - start)
                
                
                

In [ ]:
cutoffVarsYr = ['1xYr'] #, ] # ,'1xQtr''1x5Qtrs',
weatherVars  = ['precip_', 'temp_', 'precip5Days_', 'temp5Days_'] #[,]
statVarsYr   = ['annualquant_']  #,'zipQuarterquant_']
outcomeVars  = ['revenueChange',  'costChange'] # ['revenueChange'] #['incomeChange', 'revenueChange', 'costChange']#,'lnCost','lnInc','lnRev']


# 'revenueChange',  'costChange', 
# 'lnRevNormd', 'lnCostNormd'


goodsData = goodsData[~goodsData.lnRev.isna() & ~goodsData.lnCost.isna() & ~goodsData.lnCostNormd.isna()]


start = time.time()

for outcomeVar in outcomeVars:
    for weatherVar in weatherVars:
        for statVar in statVarsYr:                     
            for cutoffVar in cutoffVarsYr:
                indVar = weatherVar + statVar + cutoffVar
                supplierIndVar = 'supplier_' + weatherVar + statVar + cutoffVar
                
                
                print(outcomeVar, "~", indVar, supplierIndVar)


                # find: concurrent ; or lagged supplier data
                X = goodsData.loc[:,((goodsData.columns.str.contains(indVar) & goodsData.columns.str.contains('supplier')) |
                                  (goodsData.columns.str.contains('indQtr_'))       |
                                  (goodsData.columns.str.contains('gvkey_'))) &   
                                # (goodsData.columns.str.contains('firmQtr_')))       & 
                                  ~(goodsData.columns.str.contains('gvkey_1045'))     & # do 1 minus the FEs
                                ~(goodsData.columns.str.contains('indQtr_12013_1'))]#  &
                                # ~(goodsData.columns.str.contains('lag4')) &
                                # (goodsData.columns.str.contains('ageQtr_')) |
                                # (goodsData.columns.str.contains('sizeQtr_')) |
                                # (goodsData.columns.str.contains('profitQtr_'))) & 
                                # ~(goodsData.columns.str.contains('firmQtr_10451'))] # & # do 1 minus the FEs
                                # ~(goodsData.columns.str.contains('indQtr_12013_1'))]# & 
                                # ~(goodsData.columns.str.contains('lag2')) & 
                                 
                                
                print(X.columns[0:5])

                X = X[X.columns[(X.sum(axis = 0) >= 4)]]
                firms = goodsData['gvkey']


                y = goodsData[outcomeVar]


                ######################################
                # fit the model on this subset
                reg = linear_model.LinearRegression()
                reg.fit(X,y)


                print('Coeff: ' , reg.coef_[0:5], 'SE type (looking >2): ', findSE(X,reg,y))
                
                print(time.time() - start)
                
                print('*******************************************************************')
                


# Quarterly-Specific Effects
Look at what is going on within a quarter, for a specific zipcode. 

Note: this is effectively deprecated at this point because the zip-level effects mask the differences in the underlying weather distributions, ie some regions would likely benefit from getting warmer. If we want to do a zip-level study, we should also include some sort of threshold here.

Do this for all qtrs.

In [ ]:
cutoffVarsQtr = ['0.95','1xQtr','1x5Qtrs']# ,'1x5Qtrs'] # ,
weatherVars   = ['precip5Days_'] # ['precip5Days_','temp5Days_']
statVarQtr    = 'zipQuarterquant_'
outcomeVars   = ['revenueChange'] #['incomeChange', 'revenueChange', 'costChange']



for weatherVar in weatherVars:
    for cutoffVar in cutoffVarsQtr:
        indVar = weatherVar + statVarQtr + cutoffVar            
        for outcomeVar in outcomeVars:
            print(outcomeVar, "~", indVar)



            X = goodsData.loc[:,(goodsData.columns == indVar) | 
                              (goodsData.columns.str.contains('indQtr_')) |
                              (goodsData.columns.str.contains('gvkey_')) & 
                             ~(goodsData.columns.str.contains('gvkey_1004')) & # do 1 minus the FEs
                             ~(goodsData.columns.str.contains('indQtr_422019_4'))]


            X = X[X.columns[(X.sum(axis = 0) >= 4)]]
            firms = goodsData['gvkey']


            y = goodsData[outcomeVar]


            ######################################
            # fit the model on this subset
            reg = linear_model.LinearRegression()
            reg.fit(X,y)


            print('Coeff: ' , reg.coef_[0:5], 'SE type (looking >2): ', findSE(X,reg,y))
            print(time.time() - start)

In [ ]:
X[X.columns[(X.sum(axis = 0) >= 4)]].shape